In [1]:
from web3 import HTTPProvider
from web3 import Web3
from web3.middleware import geth_poa_middleware
import json

import operator

In [2]:
provider_url = "https://bsc-dataseed4.binance.org/"
web3 = Web3(HTTPProvider(provider_url))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

print(f"Success?: {web3.isConnected()}")

Success?: True


In [32]:
NUM_BLOCKS = 1000

## Chung
### 1. Số transaction trung bình 1000 block

In [33]:
latest_block = web3.eth.block_number
count = web3.eth.get_block_transaction_count(latest_block)
count

# result = {
#     'numberOfTransactions': count
# }
# json_obj = json.dumps(result, indent=2)
# with open('../btvn_examples/chung/chung1.json', 'w') as f:
#     f.write(json_obj)

latest_block = web3.eth.block_number
num_txs = 0
for i in range(NUM_BLOCKS):
    block_txs = web3.eth.get_block_transaction_count(latest_block - i) 
    num_txs += block_txs
    if i % 50 == 0:
        print(i)
avg_txs = num_txs / NUM_BLOCKS 
print(f"Start block: {latest_block - NUM_BLOCKS}; End block: {latest_block}")
print(f"Average transactions in {NUM_BLOCKS} blocks: {avg_txs}")

result = {
    'fromBlock': latest_block - NUM_BLOCKS,
    'toBlock': latest_block,
    'averageTransactions': avg_txs
}
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/chung/chung1.json', 'w') as f:
    f.write(json_obj)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
Start block: 21168840; End block: 21169840
Average transactions in 1000 blocks: 120.744


### 2. Cho địa chỉ token

In [36]:
token_addr = '0x7083609fCE4d1d8Dc0C979AAb8c869Ea2C873402' 
with open('../abi/erc_20.json', 'r') as f:
    abi = json.load(f)
erc20 = web3.eth.contract(address=token_addr, abi=abi)

In [38]:
# a) Crawl event
transfer_events = erc20.events.Transfer.createFilter(
    fromBlock=latest_block-NUM_BLOCKS, toBlock=latest_block).get_all_entries()
transfers = list()
for event in transfer_events:
    transfers.append(json.loads(Web3.toJSON(event)))

result = dict()
result['blocksInfo'] = {
    'startBlock': latest_block,
    'endBlock': latest_block - NUM_BLOCKS
}
result['transferEvents'] = transfers
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/chung/chung2a.json', 'w') as f:
    f.write(json_obj)

In [39]:
# b) decimals
decimal = erc20.functions.decimals().call()
print(f"Token decimal: {decimal}")

result = {
    'decimal': decimal,
}
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/chung/chung2b.json', 'w') as f:
    f.write(json_obj)

Token decimal: 18


In [40]:
# c) supply
token_total_supply = erc20.functions.totalSupply().call() / decimal 
print(f"Token supply: {token_total_supply}")

result = {
    'totalSupply': token_total_supply,
}
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/chung/chung2c.json', 'w') as f:
    f.write(json_obj)

Token supply: 8.888888884060868e+23


In [41]:
# d) user with the most transactions from
from_wallet_transfers = dict()
for transfer in transfers:
    from_wallet = transfer['args']['from']
    if from_wallet not in from_wallet_transfers:
        from_wallet_transfers[from_wallet] = 1
    else:
        from_wallet_transfers[from_wallet] += 1
# max from_wallet addr
max_from_wallet = max(from_wallet_transfers.items(), key=operator.itemgetter(1))[0]
balance = erc20.functions.balanceOf(max_from_wallet).call() / decimal

print(f"Most transactions from {max_from_wallet} with balance: {balance}")

result = {
    'wallet': max_from_wallet,
    'balance': balance,
}
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/chung/chung2d.json', 'w') as f:
    f.write(json_obj)

{'0x983BD1f0582FFf2CB9dD6068076Fe8daeDdC37cd': 2, '0xC29057b7FA51bBBE3d100255eaf07C4EaA550952': 1, '0x871DB86138Fc83BC6d3B0c2225d3810f77eF8b60': 1, '0xe2fc31F816A9b94326492132018C3aEcC4a93aE1': 2, '0x8894E0a0c962CB723c1976a4421c95949bE2D4E3': 2, '0x20ab8D465798dE05cEa422341f62C65614BCEa2C': 1, '0xAF1965202EdD913d729Bc4C5608d69F23eB66f26': 2, '0xdb1bf885c94c1B8ceE0143B8648bE097aA2623d1': 1, '0x2D26069d35fcFd8bE6D2Fb550a5D26dB99219A45': 1, '0xc590175E458b83680867AFD273527Ff58f74c02b': 2, '0x1610bc33319e9398de5f57B33a5b184c806aD217': 1, '0x396aFB2E3663f16319E6E8a0884911c70B2f7d0B': 1, '0x84d6C6DA3C53208f883F44093Db858a7A4c08cDf': 1, '0xAeae33264eaC28053CEe80feaDB8347859cf0D60': 1, '0xDD9E2fdAD01a35f5CeF8004AED7B10462B6e4028': 1, '0x33E7B46f1C7950AbAd7B256CD6580f21aD84C0BB': 1, '0x907FA5fad552Ea0792eABC67d14303C5E73B37c2': 4, '0xDd5bAd8f8b360d76d12FdA230F8BAF42fe0022CF': 2, '0x03529f8Cac4330eC8931fe62a3f1FF99aa392F5a': 1}
Most transactions from 0x907FA5fad552Ea0792eABC67d14303C5E73B37c2 wi

In [45]:
# e) user with the most transacitons to
to_wallet_transfers = dict()
for transfer in transfers:
    to_wallet = transfer['args']['to']
    if to_wallet not in to_wallet_transfers:
        to_wallet_transfers[to_wallet] = 1
    else:
        to_wallet_transfers[to_wallet] += 1
# max from_wallet addr
max_to_wallet = max(to_wallet_transfers.items(), key=operator.itemgetter(1))[0]
balance = erc20.functions.balanceOf(max_to_wallet).call() / decimal

print(f"Most transactions to {max_to_wallet} with balance: {balance}")

result = {
    'wallet': max_to_wallet,
    'balance': balance,
}
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/chung/chung2e.json', 'w') as f:
    f.write(json_obj)

Most transactions to 0xc590175E458b83680867AFD273527Ff58f74c02b with balance: 0.0


In [47]:
# f) symbol + name
symbol = erc20.functions.symbol().call()
name = erc20.functions.name().call()
print(f"Symbol: {symbol}; Name: {name}")

result = {
    'symbol': symbol,
    'name': name
}

json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/chung/chung2f.json', 'w') as f:
    f.write(json_obj)

Symbol: DOT; Name: Polkadot Token


## AAVE

In [13]:
from receipt_log_handler import EthReceiptLogHandler

lending_pool = '0x9FAD24f572045c7869117160A571B2e50b10d068'
with open('./aave_events.json') as f:
    event_abi = json.load(f)

handler = EthReceiptLogHandler()

In [14]:
event_abi_info = handler.build_list_info_event(event_abi)
print(f"event info from abi: \n {event_abi_info}")
# event hash
event_hash = [info[1] for info in event_abi_info]
print(f"event hash:\n {event_hash}")
# event subscriber
event_subscriber = {info[1] : info[0] for info in event_abi_info}
print(f"event subscribers:\n {event_subscriber}")

event info from abi: 
 [[<model.receipt_log.EventSubscriber object at 0x7fac84a215b0>, '0xc6a898309e823ee50bac64e45ca8adba6690e99e7841c45d754e2a38e9019d9b', ['reserve', 'user', 'onBehalfOf'], 'Borrow'], [<model.receipt_log.EventSubscriber object at 0x7fac847800d0>, '0xde6857219544bb5b7746f48ed30be6386fefc61b2f864cacf559893bf50fd951', ['reserve', 'user', 'onBehalfOf'], 'Deposit'], [<model.receipt_log.EventSubscriber object at 0x7fac84780fa0>, '0x3115d1449a7b732c986cba18244e897a450f61e1bb8d589cd2e69e6c8924f9f7', ['reserve', 'user', 'to'], 'Withdraw'], [<model.receipt_log.EventSubscriber object at 0x7fac84780100>, '0x4cdde6e09bb755c9a5589ebaec640bbfedff1362d4b255ebf8339782b9942faa', ['reserve', 'user', 'repayer'], 'Repay'], [<model.receipt_log.EventSubscriber object at 0x7fac84a6cf10>, '0xe413a321e8681d831f4dbccbca790d2952b56f977908e45be37335533e005286', ['collateralAsset', 'debtAsset', 'user', 'liquidator'], 'LiquidationCall']]
event hash:
 ['0xc6a898309e823ee50bac64e45ca8adba6690e99e784

In [15]:
# create filter
filter_params = {
    'fromBlock': latest_block - 2000,
    'toBlock': latest_block - 1000,
    'topics': [event_hash],
    'address': ['0x9FAD24f572045c7869117160A571B2e50b10d068']
}

event_filter = web3.eth.filter(filter_params)
events = event_filter.get_all_entries()
events
# events_list = list()
# for event in events:
#     log = handler.web3_dict_to_receipt_log(event)
#     eth_event = handler.extract_event_from_log(
#         log, event_subscriber=event_subscribers[log.topics[0]])
#     if eth_event is not None:
#         eth_event_dict = handler.eth_event_to_dict(eth_event)
#         events_list.append(eth_event_dict)

[]

## Uniswap

In [52]:
masterchefv2_addr = "0xa5f8C5Dbd5F286960b9d90548680aE5ebFf07652"
pid = 3
with open('../abi/masterchef_abi.json', 'r') as f:
    abi = json.load(f)
masterchefv2 = web3.eth.contract(abi=abi, address=masterchefv2_addr)

# 1. pool info
pool_info_array = masterchefv2.functions.poolInfo(pid).call()
pool_info = {
    'accCakePerShare': pool_info_array[0],
    'lastRewardBlock': pool_info_array[1],
    'allocPoint': pool_info_array[2],
    'totalBoostedShare': pool_info_array[3],
    'isRegular': pool_info_array[4],
}

# pool explaination
pool_explanation = {
    'accCakePerShare': '<your answer>',
    'lastRewardBlock': '<your answer>',
    'allocPoint': '<your answer>',
    'totalBoostedShare': '<your answer>',
    'isRegular': '<your answer>',
}

result = {
    'poolInfo': pool_info,
    'poolExplanation': pool_explanation,
}
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/Uniswap/uniswap1.json', 'w') as f:
    f.write(json_obj)
result
#    /// `allocPoint` The amount of allocation points assigned to the pool.
#     ///     Also known as the amount of "multipliers". Combined with `totalXAllocPoint`, it defines the % of
#     ///     CAKE rewards each pool gets.
#     /// `accCakePerShare` Accumulated CAKEs per share, times 1e12.
#     /// `lastRewardBlock` Last block number that pool update action is executed.
#     /// `isRegular` The flag to set pool is regular or special. See below:
#     ///     In MasterChef V2 farms are "regular pools". "special pools", which use a different sets of
#     ///     `allocPoint` and their own `totalSpecialAllocPoint` are designed to handle the distribution of
#     ///     the CAKE rewards to all the PancakeSwap products.
#     /// `totalBoostedShare` The total amount of user shares in each pool. After considering the share boosts.

{'poolInfo': {'accCakePerShare': 5320665111426364368,
  'lastRewardBlock': 21169582,
  'allocPoint': 1100,
  'totalBoostedShare': 4070408783102575456985145,
  'isRegular': True},
 'poolExplanation': {'accCakePerShare': '<your answer>',
  'lastRewardBlock': '<your answer>',
  'allocPoint': '<your answer>',
  'totalBoostedShare': '<your answer>',
  'isRegular': '<your answer>'}}

In [54]:
# 2. lp token address
lp_addr = masterchefv2.functions.lpToken(pid).call()
print(f"LP token address: {lp_addr}")
result = {
    'lpTokenAddress': lp_addr,
}
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/Uniswap/uniswap2.json', 'w') as f:
    f.write(json_obj)
result

LP token address: 0x58F876857a02D6762E0101bb5C46A8c1ED44Dc16


{'lpTokenAddress': '0x58F876857a02D6762E0101bb5C46A8c1ED44Dc16'}

In [61]:
with open('../abi/lp_token_abi.json', 'r') as f:
    abi = json.load(f)
lp_contract = web3.eth.contract(abi=abi, address = lp_addr)

token0 = lp_contract.functions.token0().call()
token1 = lp_contract.functions.token1().call()
print(f"token 0: {token0} \ntoken 1: {token1}")

# 3. Reserves
reserves_info = lp_contract.functions.getReserves().call()
reserve0 = reserves_info[0]
reserve1 = reserves_info[1]
print(f"reserve 0: {reserve0}\nreserve 1: {reserve1}")

# 4. Price
price0 = lp_contract.functions.price0CumulativeLast().call()  /  1e18 / reserve0
price1 = lp_contract.functions.price1CumulativeLast().call() / 1e18 / reserve1 
print(f"price 0: {price0}\nprice 1: {price1}")

token 0: 0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c 
token 1: 0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56
reserve 0: 334473682336334840716091
reserve 1: 94689283093851787856644176
price 0: 268.57212948657855
price 1: 6.481344398641888e-06


In [62]:
# 5. Events
from receipt_log_handler import EthReceiptLogHandler

with open("./lp_events.json", "r") as f:
    event_abi = json.load(f)

handler = EthReceiptLogHandler()

# info about the events based on the ABI
event_abi_info = handler.build_list_info_event(event_abi)
print(f"Event abi info: \n{event_abi_info}")

event_hash = [event_info[1] for event_info in event_abi_info]
print(f"Event hash:\n{event_hash}")

event_subscriber = {info[1]:info[0] for info in event_abi_info}
print(f"Event subsriber:\n{event_subscriber}")

Event abi info: 
[[<model.receipt_log.EventSubscriber object at 0x7f9674d55d30>, '0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822', ['sender', 'to'], 'Swap'], [<model.receipt_log.EventSubscriber object at 0x7f9674d55940>, '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', ['owner', 'spender'], 'Approval'], [<model.receipt_log.EventSubscriber object at 0x7f9674d55f10>, '0xdccd412f0b1252819cb1fd330b93224ca42612892bb3f4f789976e6d81936496', ['sender', 'to'], 'Burn'], [<model.receipt_log.EventSubscriber object at 0x7f9674d49970>, '0x4c209b5fc8ad50758f13e2e1088ba56a560dff690a1c6fef26394f4c03821c4f', ['sender'], 'Mint'], [<model.receipt_log.EventSubscriber object at 0x7f9674d49670>, '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', ['from', 'to'], 'Transfer']]
Event hash:
['0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822', '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', '0xdccd412f0b1252819cb1fd

In [67]:
#create filter
filter_params = {
    "fromBlock":latest_block-NUM_BLOCKS,
    "toBlock":latest_block,
    "topics": [event_hash],
    "address":[lp_addr]
}

logs_filter = web3.eth.filter(filter_params)  
event_logs = logs_filter.get_all_entries()  # get receipt logs of events
events_list = []
for event_log in event_logs:
    log = handler.web3_dict_to_receipt_log(event_log)
    eth_event = handler.extract_event_from_log(
        log, event_subscriber[log.topics[0]])
    if eth_event is not None:
        eth_event_dict = handler.eth_event_to_dict(eth_event)
        events_list.append(eth_event_dict)

web3.eth.uninstall_filter(logs_filter.filter_id)

print(f"Number of events: {len(events_list)}")

events_list[:2]

result = {
    'blocksInfo': {
        'startBlock': latest_block-NUM_BLOCKS,
        'endBlock': latest_block,
    },
    'events': events_list
}

json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/Uniswap/uniswap5.json', 'w') as f:
    f.write(json_obj)
result

Number of events: 1337


{'blocksInfo': {'startBlock': 21168840, 'endBlock': 21169840},
 'events': [{'type': 'event',
   'event_type': 'SWAP',
   'contract_address': '0x58f876857a02d6762e0101bb5c46a8c1ed44dc16',
   'transaction_hash': '0xcb5803c507d393d0aeea7c39e95a8ea5601376263aeb7b4fe198432f49791f41',
   'log_index': 24,
   'block_number': 21168840,
   'sender': '0x0000000000008afdacc486225455281f614843e7',
   'to': '0x668c440d71118ca449c556ff4bfcfc64ca205ff9',
   'amount0In': '68723789917271461',
   'amount1In': '0',
   'amount0Out': '0',
   'amount1Out': '19283970876497865916'},
  {'type': 'event',
   'event_type': 'TRANSFER',
   'contract_address': '0x58f876857a02d6762e0101bb5c46a8c1ed44dc16',
   'transaction_hash': '0x5d856a98f6f4856bab07a6d9f618d3bf6a7e33bd3feb3e8e5730e436aab2e901',
   'log_index': 129,
   'block_number': 21168840,
   'from': '0x3f94533a6bca463ad098e9ba2206133916f92e19',
   'to': '0xa738c23b69c76c6dc6a2ff8a3af81c3685a93658',
   'value': '6746000000000000000'},
  {'type': 'event',
   'ev

In [70]:
# 6. Transactions data
tx_hashes_list = [event['transaction_hash'] for event in events_list]
# tx_list = [web3.eth.get_transaction(tx_hash) for tx_hash in tx_hashes_list]
tx_list = list()
for i, tx_hash in enumerate(tx_hashes_list):
    tx = web3.eth.get_transaction(tx_hash)
    tx_list.append(tx)
    if i%10 == 0:
        print(i)

tx_list[0]

result = {
    'blocksInfo': {
        'startBlock': latest_block-NUM_BLOCKS,
        # 'startBlock': latest_block-10,
        'endBlock': latest_block,
    },
    'transactions': tx_list
}
print(f"Number of transactions: {len(tx_list)}")

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330


1337

In [89]:
# process data for saving in JSON 
from copy import deepcopy

transactions_list = [dict(tx) for tx in tx_list]
transactions_list_2 = list()
transactions_list
for t in transactions_list:
    t2 = deepcopy(t)
    t2.pop('blockHash', None)
    t2.pop('hash', None)
    t2.pop('r', None)
    t2.pop('s', None)
    t2['blockHash'] = t['blockHash'].hex()
    t2['hash'] = t['hash'].hex()
    t2['r'] = t['r'].hex()
    t2['s'] = t['s'].hex()
    transactions_list_2.append(t2)

result = {
    'blocksInfo': {
        'startBlock': latest_block-NUM_BLOCKS,
        'endBlock': latest_block,
    },
    'transactions': transactions_list_2
}
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/Uniswap/uniswap6.json', 'w') as f:
    f.write(json_obj)

[{'blockNumber': 21168840,
  'from': '0x00004BD7cDFAD64B42726041ab2847ed6D96feFA',
  'gas': 5000007,
  'gasPrice': 12000000000,
  'input': '0x00000003000000000000000000030d40bb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c0000000000000000000226f7610aee732ac2a928abffaa49d655ad2cadb40c220000000000000000000026f7668c440d71118ca449c556ff4bfcfc64ca205ff90000000000000000000026f758f876857a02d6762e0101bb5c46a8c1ed44dc160000000000000000000000001fed8951d852b88930557b681409b334e1089343000000000000000000000000e9e7cea3dedca5984780bafc599bd69add087d56',
  'nonce': 2145,
  'to': '0x0000000000008AfdAcc486225455281F614843e7',
  'transactionIndex': 4,
  'value': 0,
  'type': '0x0',
  'v': 147,
  'blockHash': '0x6b324ca779892f96f52f350a0c4dc44ff4119b47d6e0570aaa90a037bd87c133',
  'hash': '0xcb5803c507d393d0aeea7c39e95a8ea5601376263aeb7b4fe198432f49791f41',
  'r': '0xa4a766f3a72395b644c3bcff2310f27f83ebccb6d4417af1652194b48c5a97da',
  's': '0x5a7f6741db372e069e3a36a16a8ff68874ffc972ca11696696c004ee6d3b4650'},
 {'b

In [66]:
# 7. User with the most transactions from
from_user_tx = dict()
for tx in tx_list:
    from_user = tx['from']
    if from_user not in from_user_tx:
        from_user_tx[from_user] = 1
    else:
        from_user_tx[from_user] += 1

max_from_user = max(from_user_tx.items(), key=operator.itemgetter(1))
max_from_user_addr = max_from_user[0]
print(f"User {max_from_user[0]} has the most transactions: {max_from_user[1]}")

result = {
    'wallet': max_from_user_addr[0],
    'numberOfTransactions': max_from_user[1]
}
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/Uniswap/uniswap7.json', 'w') as f:
    f.write(json_obj)

User 0xADC31a85C01aeBA202Df01adc392a7c6b8D56916 has the most transactions: 32


In [67]:
# 8. User info
user_info = masterchefv2.functions.userInfo(pid, max_from_user_addr).call()
user_info

result = {
    'userInfo': user_info,
    'infoExplanation': {
        '<field1>': '<your answer>',
        '<field2>': '<your answer'
    }
}
json_obj = json.dumps(result, indent=2)
with open('../btvn_examples/Uniswap/uniswap8.json', 'w') as f:
    f.write(json_obj)

[0, 0]

User info gồm:
- "amount": số lượng LP token mà user đó cung cấp
- "rewardDebt": để tính correct amount cho reward
pending reward = (user share * pool.accCakePerShare) - user.rewardDeb    ///